# Demonstration usage of pyessv library

## Setup

### Import libs.

In [ ]:
import os

import pyessv

### Set constants.

In [ ]:
_VOCAB_DOMAIN = u"ipsl.hermes"
_VOCAB_IO_DIR = os.getenv("PYESSV_HOME")
_TERMSET_COMPUTE_NODE = u"compute_node"

## Partition creation

In [ ]:
# Create vocab partition.
partition = pyessv.create_partition(
    _VOCAB_DOMAIN,
    _VOCAB_IO_DIR
)

In [ ]:
# Show partitions.
pyessv.show_partitions()

## Partition cache

### Load

In [ ]:
# Loads terms from file system into memory.
partition.load()

### Counts

In [ ]:
# Cache count - all terms in all termsets;
assert partition.get_count() == 656

# Cache count - terms within a single termset;
assert partition.get_count(_TERMSET_COMPUTE_NODE) == 9

# Cache count - single term;
assert partition.get_count(_TERMSET_COMPUTE_NODE, "tgcc") == 1

### Access

In [ ]:
# Cache access - all termsets;
termsets = partition.query()
assert isinstance(termsets, dict)

# Cache access - single termset;
termset = partition.query(_TERMSET_COMPUTE_NODE)
assert isinstance(termset, list)

# Cache access - single term.
term = partition.query(_TERMSET_COMPUTE_NODE, u"tgcc")
assert isinstance(term, pyessv.Term)

### Listing

In [ ]:
# Cache listing - all termsets;
partition.show()

In [ ]:
# Cache listing - single termset;
partition.show(_TERMSET_COMPUTE_NODE)

In [ ]:
# Cache listing - single term;
partition.show(_TERMSET_COMPUTE_NODE, u"tgcc")

## Term management

### Creation

In [ ]:
# Use partition to create new term.
term = partition.create(_TERMSET_COMPUTE_NODE, u"xxx")
assert isinstance(term, pyessv.Term)

### Validation

In [ ]:
# Invalid term - kind has invalid characters.
invalid_term = partition.create(u"&!@", u"xxx")

assert invalid_term.is_valid == False
assert len(invalid_term.errors) == 1

invalid_term.show_errors()

In [ ]:
# Invalid term - name has invalid characters.
invalid_term = partition.create(_TERMSET_COMPUTE_NODE, u"x&!")

assert invalid_term.is_valid == False
assert len(invalid_term.errors) == 1

invalid_term.show_errors()

In [ ]:
# Invalid term - kind & name have invalid characters.
invalid_term = partition.create(u"&!@", u"x&!")

assert invalid_term.is_valid == False
assert len(invalid_term.errors) == 2

invalid_term.show_errors()

### Encoding

In [ ]:
# Encode as python dictionary.
as_dict = term.as_dict()
assert isinstance(as_dict, dict)

In [ ]:
# Encode as JSON.
as_json = term.as_json()
assert isinstance(as_json, str)

### Persistence

In [ ]:
# Verify term is not yet cached in a partition.
assert term.is_cached == False

# Verify term is not yet written to file system.
assert term.is_written == False

In [ ]:
# Saving a term writes to cache & file system.
term.save()

In [ ]:
# Verify saved term is cached in a partition.
assert term.is_cached == True

# Verify saved term is written to file system.
assert term.is_written == True

## Term Governance

In [ ]:
# What is initial governance status ?
term.status

In [ ]:
# Accept.
term.accept()

In [ ]:
# Reject.
term.reject()

In [ ]:
# Deprecate.
term.deprecate()

In [ ]:
partition.load()
print _TERMSET_COMPUTE_NODE
print partition.get_count(_TERMSET_COMPUTE_NODE)

In [ ]:
# Destroy.
assert partition.get_count(_TERMSET_COMPUTE_NODE) == 8
assert partition.get_count(_TERMSET_COMPUTE_NODE, "xxx") == 1

term.destroy()

assert partition.get_count(_TERMSET_COMPUTE_NODE) == 7
assert partition.get_count(_TERMSET_COMPUTE_NODE, "xxx") == 0

## Term Parsing

In [ ]:
# Use partition to create new test term.
term = partition.create(_TERMSET_COMPUTE_NODE, u"yyy")

# Set term alternative name.
term.alternative_name = u"zzz"

# Set term alias.
term.aliases.append("YYY-1")

# Save term.
partition.save(term)

In [ ]:
# Parse a name that matches. 
name = "yyy"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, name)
assert name == parsed_name

In [ ]:
# Parse a name that almost matches. 
name = "yYY"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, name)
assert name != parsed_name
assert parsed_name == "yyy"

In [ ]:
# Parse an alternative name.
alt_name = "zzz"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, alt_name)
assert alt_name != parsed_name
assert parsed_name == "yyy"

In [ ]:
# Parse an alias.
alias = "YYY-1"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, alias)
assert alias != parsed_name
assert parsed_name == "yyy"

In [ ]:
# Destroy test term.
term.destroy()

## Term Associations

In [ ]:
# TODO !!!

## Term Grouping

In [ ]:
# TODO 
# Write groups of terms to file system in blocks so as to streamline downloading in UI !!!